# Sample 2: Scopes & Summaries

Originally by: Kaspar Sakmann (https://github.com/ksakmann)

A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [1]:
import tensorflow as tf
import numpy as np

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
with tf.name_scope("input_var"):
    x = tf.placeholder(tf.float32, [None, n_input])
with tf.name_scope("input_target"):
    y = tf.placeholder(tf.float32, [None, n_classes])
with tf.name_scope("keep_prob"):
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [3]:
# Create some wrappers for simplicity
def conv2d(x, W, b, name, strides=1):
    # Conv2D wrapper, with bias and relu activation
    with tf.name_scope(name):
        x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, name, k=2):
    # MaxPool2D wrapper
    with tf.name_scope(name):
        pool = tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')
    return pool


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    with tf.name_scope("input_img"):
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['weights_conv1'], biases['bias_conv1'], "conv1")
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1,"pooling_1", k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['weights_conv2'], biases['bias_conv2'], "conv2")
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, "pooling_2", k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    with tf.name_scope("fully_connected"):
        fc1 = tf.reshape(conv2, [-1, weights['weights_fully'].get_shape().as_list()[0]])
        fc1 = tf.add(tf.matmul(fc1, weights['weights_fully']), biases['bias_fully'])
        fc1 = tf.nn.relu(fc1)
        # Apply Dropout
        fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    with tf.name_scope("class_prediction"):
        out = tf.add(tf.matmul(fc1, weights['weights_out']), biases['bias_out'])
    return out

In [4]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10
log_dir = "/tmp/tensorflow_logs"

In [5]:
def variable_summaries(var, name):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope(name):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)


In [6]:
# Store layers weight & bias
with tf.name_scope("weights"):
    weights = {
        # 5x5 conv, 1 input, 32 outputs
        'weights_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32]), name="weights_conv1"),
        # 5x5 conv, 32 inputs, 64 outputs
        'weights_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64]), name="weights_conv2"),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'weights_fully': tf.Variable(tf.random_normal([7*7*64, 1024]), name="weights_fully"),
        # 1024 inputs, 10 outputs (class prediction)
        'weights_out': tf.Variable(tf.random_normal([1024, n_classes], name="weights_out"))
    }

with tf.name_scope("biases"):
    biases = {
        'bias_conv1': tf.Variable(tf.random_normal([32]), name="bias_conv1"),
        'bias_conv2': tf.Variable(tf.random_normal([64]), name="bias_conv2"),
        'bias_fully': tf.Variable(tf.random_normal([1024]), name="bias_fully"),
        'bias_out': tf.Variable(tf.random_normal([n_classes]), name="bias_out")
    }

#set advanced variable summaries logging
for lbl, weight in weights.items():
    variable_summaries(weight, lbl)
for lbl, bias in biases.items():
    variable_summaries(bias, lbl)

# Construct model
with tf.name_scope("model"):
    pred = conv_net(x, weights, biases, keep_prob)

with tf.name_scope("learning"):
    # Define loss and optimizer
    with tf.name_scope("cost"):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

    with tf.name_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    with tf.name_scope("correct_pred"):
        correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", accuracy)


<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [7]:
# Launch the graph
with tf.Session() as sess:
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')

    # Initializing the variables
    init = tf.global_variables_initializer()    
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        _, summary = sess.run([optimizer, merged ], feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        
        train_writer.add_summary(summary, step*batch_size)
        
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))
    print ("Run the command line:\n" \
          "--> tensorboard --logdir="+log_dir+" " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Iter 1280, Minibatch Loss= 19412.835938, Training Accuracy= 0.32812
Iter 2560, Minibatch Loss= 7671.116699, Training Accuracy= 0.57031
Iter 3840, Minibatch Loss= 6444.958008, Training Accuracy= 0.64844
Iter 5120, Minibatch Loss= 3239.897217, Training Accuracy= 0.78906
Iter 6400, Minibatch Loss= 3844.964111, Training Accuracy= 0.76562
Iter 7680, Minibatch Loss= 3099.986328, Training Accuracy= 0.88281
Iter 8960, Minibatch Loss= 4070.644531, Training Accuracy= 0.81250
Iter 10240, Minibatch Loss= 2097.285156, Training Accuracy= 0.87500
Iter 11520, Minibatch Loss= 1827.633911, Training Accuracy= 0.86719
Iter 12800, Minibatch Loss= 2270.322266, Training Accuracy= 0.89844
Iter 14080, Minibatch Loss= 1045.293701, Training Accuracy= 0.89844
Iter 15360, Minibatch Loss= 1472.544922, Training Accuracy= 0.91406
Iter 16640, Minibatch Loss= 662.940552, Training Accuracy= 0.92969
Iter 17920, Minibatch Loss= 1282.223145, Training Accuracy= 0.94531
Iter 19200, Minibatch Loss= 1474.467285, Training Accur

Iter 157440, Minibatch Loss= 410.108246, Training Accuracy= 0.94531
Iter 158720, Minibatch Loss= 170.529190, Training Accuracy= 0.96875
Iter 160000, Minibatch Loss= 139.320374, Training Accuracy= 0.97656
Iter 161280, Minibatch Loss= 156.360672, Training Accuracy= 0.97656
Iter 162560, Minibatch Loss= 370.376770, Training Accuracy= 0.92969
Iter 163840, Minibatch Loss= 406.705078, Training Accuracy= 0.96094
Iter 165120, Minibatch Loss= 151.089249, Training Accuracy= 0.99219
Iter 166400, Minibatch Loss= 15.867722, Training Accuracy= 0.99219
Iter 167680, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 168960, Minibatch Loss= 54.399078, Training Accuracy= 0.98438
Iter 170240, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 171520, Minibatch Loss= 260.655670, Training Accuracy= 0.97656
Iter 172800, Minibatch Loss= 143.320129, Training Accuracy= 0.96875
Iter 174080, Minibatch Loss= 55.452354, Training Accuracy= 0.97656
Iter 175360, Minibatch Loss= 87.394966, Training Accura